In [ ]:
import os, sys, glob, scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nilearn, nibabel, nltools
from nltools.data import Brain_Data
from nilearn.plotting import plot_glass_brain, plot_img, plot_stat_map
from nistats.reporting import get_clusters_table

In [ ]:
def get_clusters(image, threshold = 0, extent_threshold = 0, sort_by_size = False, include_peaks_only = True):
    clusters = get_clusters_table(image, threshold, cluster_threshold = extent_threshold)
    clusters = clusters.rename(columns = {'Cluster ID':'ID', 'Cluster Size (mm3)':'size',
                                          'Peak Stat':'peak_value'})
    if include_peaks_only:
        peaks = clusters.copy()
        peaks = peaks.loc[peaks['ID'].apply(lambda x: not str(x)[-1].isalpha()),:].reset_index(drop=True)
#         peaks = peaks.sort_values(by = 'size', ascending = False).reset_index(drop=True)
    if sort_by_size:
        clusters = clusters.sort_values(by = 'size', ascending = False).reset_index(drop=True)
    return clusters, peaks

In [ ]:
results_dir = os.path.realpath('../../../Results/voxelwise_ISRSA/nifti')
results_dir

## Select run and model

In [ ]:
run = 3
filter_TR = False
TR_start = 1
TR_end = 711
model = 'ideology_IUS'
thresholds = ['fdr-0.05']

In [ ]:
if filter_TR:
    predictor_list = glob.glob(results_dir + '/run-%i_TRs-%i-%i_model-%s/*fdr*.nii.gz'%(run,TR_start,TR_end,model))
else:
    predictor_list = glob.glob(results_dir + '/run-%i_model-%s/*fdr*.nii.gz'%(run,model))
predictor_list = [i.split('/')[-1][19:-30] for i in predictor_list]
predictor_list = [i for i in predictor_list if i not in [
    '(Intercept)','same_communityTRUE', 'scale(scan_day_distance)',
    'scale(age_distance)','same_genderTRUE','same_undergradTRUE']]
predictor_list = [i for i in predictor_list if '*' not in i]
predictor_list = sorted(predictor_list)
predictor_list

## Plot results for predictor

In [ ]:
for predictor in predictor_list:
    print('*****\n%s'%predictor)
    for threshold in thresholds:
        print('\n%s\n'%threshold)
        fname = results_dir + '/run-%i_model-%s/dyad_ISC_regressor-%s_beta-thr-pval-%s.nii.gz'%(
            run, model, predictor, threshold)
        print(fname)
        plot_stat_map(fname)
        plt.show()
        print('\n')
    print('\n\n')

## Report clusters

In [ ]:
beta_map_thresh = nilearn.image.load_img(fname)

In [ ]:
clusters, peaks = get_clusters(beta_map_thresh, threshold = 0, extent_threshold = 10, include_peaks_only = True)
clusters